In [ ]:
import numpy as np
import pandas as pd
from pandas.core.internals.construction import to_arrays
from prompt_toolkit.utils import to_int
from pulp import *

In [ ]:
def find_loops(edges):
    """
    Find all closed loops starting and ending at node 0 in a directed graph.

    Args:
        edges: List of tuples representing directed edges (from_node, to_node)

    Returns:
        List of lists, where each sublist contains tuples forming a closed loop from 0 to 0
    """
    # Create adjacency list for faster lookup
    adj = {}
    for start, end in edges:
        if start not in adj:
            adj[start] = []
        adj[start].append(end)

    def find_paths(current, path, visited):
        """
        Recursive helper function to find all paths from current node back to 0.
        """
        if current == 'Depot' and len(path) > 0:  # Found a loop back to 0
            return [path[:]]

        if current not in adj:  # Dead end
            return []

        paths = []
        for next_node in adj[current]:
            if (current, next_node) not in visited:  # Avoid using same edge twice
                path.append((current, next_node))
                visited.add((current, next_node))
                paths.extend(find_paths(next_node, path, visited))
                path.pop()
                visited.remove((current, next_node))
        return paths

    # Start DFS from node 0
    all_loops = find_paths('Depot', [], set())

    # Filter out non-minimal loops (loops that contain other loops)
    minimal_loops = []
    for loop in all_loops:
        # Check if this loop contains any other loop
        is_minimal = True
        visited_nodes = {node for edge in loop for node in edge}
        for other_loop in all_loops:
            if loop != other_loop:
                other_nodes = {node for edge in other_loop for node in edge}
                if other_nodes.issubset(visited_nodes) and len(other_loop) < len(loop):
                    is_minimal = False
                    break
        if is_minimal:
            minimal_loops.append(loop)

    return minimal_loops

## Cost matrix

In [ ]:
cost_matrix = pd.read_csv('../costs/costMatrixDistance.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDistance-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial-2020.csv', index_col=0)

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(float)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(str)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(str)

## Shop demands

In [ ]:
shop_demands = pd.read_csv('../2015_shop_locations.csv').set_index('id')[['demand(kg)', 'stage']].astype(float)
shop_demands = shop_demands.loc[shop_demands['stage'].isin([1, 2, 3, 4]), 'demand(kg)'] # Choose the stages to visit
shops = shop_demands.index
nodes = shops.copy()
nodes = np.append('Depot', nodes)

shop_demands.shape

## Vehicle routing problem

In [ ]:
# Vehicle capacity
van_capacity = 300

In [ ]:
# Creates a list of tuples containing all the possible routes for transport
A = [(i,j) for i in nodes for j in nodes if i!=j]

In [ ]:
A

In [ ]:
# A set of variables x is created to contain the vehicle routes variables (binary var)
x = {}
for a in A:
    x[a] = LpVariable("x(%s,%s)" %a, cat=LpBinary)

# A set of variables u is created to contain the continuous delivered quantity
u = {}
for shop in shops:
    u[shop] = LpVariable("u(%s)" %shop, shop_demands[shop], van_capacity)

In [ ]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("CVRP_Problem",LpMinimize)

In [ ]:
# The objective function is added to 'prob' first
prob += lpSum(cost_matrix.loc[i,j]*x[i,j] for (i,j) in A)

In [ ]:
#Constraint 1 : imposes that exactly one arc enters each customer node
one_visit_out = {}
for shop in shops:
    one_visit_out[shop] = lpSum([x[shop,node] for node in nodes if shop!=node]) == 1.0
    prob += one_visit_out[shop]

In [ ]:
#Constraint 2 : imposes that exactly one arc leaves each customer node
one_visit_in = {}
for shop in shops:
    one_visit_in[shop] = lpSum([x[node,shop] for node in nodes if node!=shop]) == 1.0
    prob += one_visit_in[shop]

In [ ]:
#Constraint 4 : imposes the amount of flow that leaves the depot should be identical
#with the flow returns to the depot
depot_node = lpSum([x['Depot',shop] for shop in shops]) == lpSum([x[shop,'Depot'] for shop in shops])
prob += depot_node

In [ ]:
#Constraint 5 : vehicle capacity constraint and sub tour elimination constraint
sub_tours_cap = {}
for shop_out in shops:
    for shop_in in shops:
        if shop_out != shop_in:
            sub_tours_cap[shop_out,shop_in] = u[shop_out] - u[shop_in] + van_capacity * x[shop_out,shop_in] <= van_capacity - shop_demands[shop_in]
            prob += sub_tours_cap[shop_out,shop_in]

In [ ]:
# The problem is solved using PuLP's choice of Solver (ici Cplex, retourne 1 si résolu, -1 sinon)
prob.solve(GUROBI(msg=True, timeLimit=600))

In [ ]:
# The status of the solution is printed to the screen (optimal ou pas)
print("Status:", LpStatus[prob.status])

In [ ]:
# Each of the variables is printed with it's resolved optimum value
for key,val in x.items():
    if val.varValue==1.0:
        print(val,"=",val.varValue)

In [ ]:
# Print bounds
print("Lower bound = ", value(prob.solverModel.ObjBound))
print("Upper bound = ", value(prob.solverModel.ObjVal))

In [ ]:
# Afficher les acrs correspondant à la solution
active_arcs = [key for key,val in x.items() if val.varValue >=0.9 ]
find_loops(active_arcs)


In [ ]:
# Each of the variables is printed with it's resolved optimum value
for key,val in u.items():
    print(val,"=",val.varValue)

In [ ]:
# Convert the list of active arcs to a DataFrame
active_arcs_df = pd.DataFrame(active_arcs, columns=['From', 'To'])

# Save the DataFrame to a CSV file
active_arcs_df.to_csv('../routings/routing_optimal_2015_stage_4_Distance.csv', index=False)